In [ ]:
import random
import calendar
import time
import openpyxl
import hashlib
from Crypto import Random
from Crypto.Cipher import AES
import base64
from base64 import b64encode, b64decode


def chash(s):
  hash = hashlib.sha256(s.encode()).hexdigest()
  return hash


class AESCipher(object):
  def __init__(self, key):
    self.block_size = AES.block_size
    self.key = hashlib.sha256(key.encode()).digest()

  def encrypt(self, plain_text):
    plain_text = self.__pad(plain_text)
    iv = Random.new().read(self.block_size)
    cipher = AES.new(self.key, AES.MODE_CBC, iv)
    encrypted_text = cipher.encrypt(plain_text.encode())
    return b64encode(iv + encrypted_text).decode("utf-8")

  def decrypt(self, encrypted_text):
    encrypted_text = b64decode(encrypted_text)
    iv = encrypted_text[:self.block_size]
    cipher = AES.new(self.key, AES.MODE_CBC, iv)
    plain_text = cipher.decrypt(encrypted_text[self.block_size:]).decode("utf-8")
    return self.__unpad(plain_text)

  def __pad(self, plain_text):
    number_of_bytes_to_pad = self.block_size - len(plain_text) % self.block_size
    ascii_string = chr(number_of_bytes_to_pad)
    padding_str = number_of_bytes_to_pad * ascii_string
    padded_plain_text = plain_text + padding_str
    return padded_plain_text

  @staticmethod
  def __unpad(plain_text):
    last_character = plain_text[len(plain_text) - 1:]
    return plain_text[:-ord(last_character)]


def bxor(ba1, ba2):
    return bytes([_a ^ _b for _a, _b in zip(ba1, ba2)])


def server_side_registration(id, rpw, tid):

  # generate random secret y
  Y = str (random.randrange(1000000, 9999999, 1))
 
	# get server secrets 
  file = open("server_secrets.txt", "r")
  a = file.readline()
  y = file.readline()
  z = file.readline()
  ks = file.readline()
  x = x[:-1]
  y = y[:-1]
  z = z[:-1]

  aes_obj = AESCipher(ks)

  p_id = aes_obj.encrypt(Y + id)

  a = chash(x + bxor(id.encode(), y.encode()).decode() + z)
  b_dash = bxor(a.encode(), rpw.encode()).decode()
  c_dash = bxor(Y.encode(), rpw.encode()).decode()
  d_dash = chash(a + Y + id)

  wb_obj = openpyxl.load_workbook("server.xlsx")
  sheet_obj = wb_obj.active
  rows = sheet_obj.max_row
  params = [tid, p_id]

  for i in range(1, rows + 1):
    if sheet_obj.cell(row = i, column = 1).value == None:
      ind = int (1)
      for x in params:
        message_bytes = x.encode()
        base64_bytes = base64.b64encode(message_bytes)
        sheet_obj.cell(row = i, column = ind).value = base64_bytes.decode()
        ind += 1   
      break
  wb_obj.save("server.xlsx")

  return [b_dash, c_dash, d_dash]


def client_side_registration():

	# input id password for registration
  print ("--> Enter ID: ", end = '')
  id_ = input()
  print ("--> Enter Password: ", end = '')
  password_ = input()
  print ("--> Confirm Password: ", end = '')
  confirm_password_ = input()

	# check condition
  if password_ != confirm_password_ :
    print ("** Password & Confirm Password don't match! Please try again!")
    return

	# generate random secret m
  sec = str (random.randrange(2874284, 764517642548164562, 1))

	# calculate rpw
  rpw = chash(password_ + str (sec))
  R = str (random.randrange(2874284, 764517642548164562, 1))
  rpw_dash = bxor(rpw.encode(), R.encode()).decode()

  tid = str(random.randrange(1, 20, 1))

  params = server_side_registration(id_, rpw_hdash, tid)

  params[0] = b_dash
  params[1] = c_dash
  params[2] = d_dash

  sigma = str (random.randrange(2874284, 764517642548164562, 1))
  tou = str (random.randrange(2874284, 764517642548164562, 1))
  bio = sigma ^ tou

  b = bxor(bxor(b_dash.encode(), rpw_dash.encode()), chash(sigma + id_ + rpw).encode()).decode()
  c = bxor(c_dash.encode(), chash(a + sigma + id_ + rpw).encode()).decode()
  d = chash(sigma + rpw + d_dash)
  e = bxor(sec.encode(), chash(sigma + id_ + password_).encode()).decode()

  wb_obj = openpyxl.load_workbook("users.xlsx")
  sheet_obj = wb_obj.active
  rows = sheet_obj.max_row
  ok = True
  for i in range(1, rows + 1):
    if sheet_obj.cell(row = i, column = 6).value == id_:
      ok = False
      break

  if ok == True:
    params = [tid, b, c, d, e, tou]
    for i in range(1, rows + 1):
      if sheet_obj.cell(row = i, column = 3).value == None:
        ind = int (1)
        for x in params:
          message_bytes = x.encode()
          base64_bytes = base64.b64encode(message_bytes)
          sheet_obj.cell(row = i, column = ind).value = base64_bytes.decode()
          ind += 1   
        sheet_obj.cell(row = i, column = ind).value = id_
        break
    wb_obj.save("users.xlsx")
    print ("** Succefully registered!")

  else:
    print ("** Username already exists!")


def server_side_auth(tid, f, rr1, g, ts):
  gmt = time.gmtime()
  ts_server = str (calendar.timegm(gmt))
  if (int (ts_server) - int (ts)) > 1:
    print ("** Something went wrong!")
    return []

  # get server secrets 
  file = open("server_secrets.txt", "r")
  x = file.readline()
  y = file.readline()
  z = file.readline()
  ks = file.readline()
  x = x[:-1]
  y = y[:-1]
  z = z[:-1]

  pid = hash_map[tid]

  aes_obj = AESCipher(ks)

  extracted = aes_obj.decrypt(pid)

  id = extracted[7:]
  Y = ""
  for i in range (0, 7):
    Y += extracted[i]

  a = chash(x + bxor(id.encode(), y.encode()).decode() + z)
  rand1 = bxor(rr1.encode(), chash(Y + a + id + ts).encode()).decode()
  g_n = chash(a + bxor(a.encode(), Y.encode()).decode() + Y + rand1 + id + ts)

  if g != g_n:
    print ("** Something went wrong!")
    return []

  ts_server = str (calendar.timegm(gmt))
  rand2 = str (random.randrange(2874284, 764517642548164562, 1))
  tid_n = str(random.randrange(1, 20, 1))

  rr2 = bxor(chash(ts_server + a + Y + id).encode(), chash(rand2 + ts_server)).decode()

  t = chash(ts_server + bxor(a.encode(), Y.encode()).decode() + rr2 + a + Y + id)
  
  temp = bxor(tid.encode(), ts.encode()).decode()
  temp = bxor(temp.encode(), ts_server.encode()).decode()
  temp = bxor(temp.encode(), id.encode()).decode()
  temp = bxor(temp.encode(), a.encode()).decode()
  temp = bxor(temp.encode(), Y.encode()).decode()
  temp = bxor(temp.encode(), chash(rand2 + ts_server).encode()).decode()
  temp = temp + rand1
  temp = chash(temp)
  sk = temp

  skv = chash(ts_server + sk + rand1)

  tid_ = bxor(tid_n.encode(), chash(tid + ts_server + sk).encode()).decode()

  return [tid_, t, rr2, skv, ts_server]

def client_side_auth():
  print ("--> Enter ID: ", end = '')
  id_ = input()
  print ("--> Enter Password: ", end = '')
  password_ = input()

  bio = input()

  wb_obj = openpyxl.load_workbook("users.xlsx")
  sheet_obj = wb_obj.active
  rows = sheet_obj.max_row
  ok = False
  row_no = -1
  for i in range(1, rows + 1):
    if sheet_obj.cell(row = i, column = 7).value == id_:
      row_no = i
      ok = True
      break

  if ok == False:
    print ("** ID doesn't exist!")
    return

  tid = base64.b64decode(sheet_obj.cell(row = row_no, column = 1).value.encode()).decode()
  b = base64.b64decode(sheet_obj.cell(row = row_no, column = 2).value.encode()).decode()
  c = base64.b64decode(sheet_obj.cell(row = row_no, column = 3).value.encode()).decode()
  d = base64.b64decode(sheet_obj.cell(row = row_no, column = 4).value.encode()).decode()
  e = base64.b64decode(sheet_obj.cell(row = row_no, column = 5).value.encode()).decode()
  tou = base64.b64decode(sheet_obj.cell(row = row_no, column = 5).value.encode()).decode()

  sigma = int (tou) ^ int (bio)
  sigma = str (sigma)

  sec = bxor(e.encode(), chash(sigma + id_ + password_).encode()).decode()
  rpw = chash(password_ + sec)

  a = bxor(b.encode(), chash(sigma + id + rpw).encode()).decode()
  Y = bxor(c.encode(), chash(a + sigma + id_ + rpw).encode()).decode()
  d_n = chash(sigma + rpw + chash(a + Y + id_))

  if d_n != d:
    print ("** Something went wrong!")
    return
  
  gmt = time.gmtime()
  ts = str (calendar.timegm(gmt))
  rand1 = str (random.randrange(2874284, 764517642548164562, 1))

  temp = bxor(id_.encode(), a.encode()).decode()
  temp = bxor(temp.encode(), ts.encode()).decode()
  temp = chash(temp)
  f = bxor(Y.encode(), temp.encode()).decode()
  temp = bxor(a.encode(), id_.encode()).decode()
  f = bxor(f.encode(), temp.encode()).decode()
  rr1 = bxor(chash(rand1 + ts).encode(), chash(Y + a + id_ + ts).encode()).decode()
  g = chash(a + temp + Y + rand1 + id_ + ts)

  params = server_side_auth(tid, f, rr1, g, ts)

  if (len(params) == 0):
    return
  
  gmt = time.gmtime()
  n_ts = str (calendar.timegm(gmt))

  if (int (n_ts) - int (params[4])) > 1:
    print ("** Something went wrong!")
    return

  t = chash(params[4] + bxor(a.encode(), Y.encode()).decode() + params[2] + a + Y + id)

  if t != params[1]:
    print ("** Something went wrong!")
    return

  rand2 = bxor(params[2].encode(), chash().encode(params[4] + a + Y + id)).decode()

  temp = bxor(tid.encode(), ts.encode()).decode()
  temp = bxor(temp.encode(), params[4].encode()).decode()
  temp = bxor(temp.encode(), id.encode()).decode()
  temp = bxor(temp.encode(), a.encode()).decode()
  temp = bxor(temp.encode(), Y.encode()).decode()
  temp = bxor(temp.encode(), rand2.encode()).decode()
  temp = temp + rand1
  temp = chash(temp)
  sk = temp

  skv = chash(params[4] + sk + rand1)

  tid_n = bxor(params[0].encode(), chash(tid + params[4] + sk).encode()).decode()

  message_bytes = sk.encode()
  base64_bytes = base64.b64encode(message_bytes)
  sheet_obj.cell(row = row_no, column = 8).value = base64_bytes.decode()
  wb_obj.save("users.xlsx")
  
  message_bytes = skv.encode()
  base64_bytes = base64.b64encode(message_bytes)
  sheet_obj.cell(row = row_no, column = 9).value = base64_bytes.decode()
  wb_obj.save("users.xlsx")

  update tid with tid_n in "server.txt"

  print ("* Successfully Logged in!")

client_side_registration()
client_side_auth()

--> Enter ID: 1
--> Enter Password: 1
--> Confirm Password: 1
['AQBQUwJXVgsEVVNRBQACVlAAUAJTXAUCBwJQBgQIAwZcAlFQBVtUAAEBAQIHWwxTVQUDBlFfBwBdU1cEAFxQDg==', 'UwdRAVUNU1UJUFVWBFBZAgZcCw4FB1EGA1IHCVRXVgAFVA4PBlECVgMCBldQAFZSAwEDUwZQClMDX1NUBFVQAQ==', 'AVNWBQcGBglTAwEGVgJTAQgCA1c=', 'X1NaDQsPU1lUV1daUwIHAFIDXVwGVVYPV1QDB1ILAwVSV1xVBldRAAABAFZTAAMGBFNVVgNRBwEFAA9XBwUIVw==', 'ZjlkZjY4MmJmYjFlMjRmNjVlOTQwZDg3NWM2OGU4YjZjZjBhOWViMGFhODIxNDNiYjc2ZDEyOGJlOGE0MGVmZQ==', 'NjEyMjEzMQ==', 'DQcJUwlVDQACAQENVQZTAVBTDQM=']
